In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install catboost

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

from sklearn.metrics import classification_report, accuracy_score 
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, matthews_corrcoef
from sklearn.metrics import confusion_matrix

In [4]:
#importing total X_train(features)
X = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/X.csv')

In [5]:
#importing total Y_train(target)
y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/y.csv')

In [6]:
#model building and fitting
catboost_clf = CatBoostClassifier()
catboost_clf.fit(X, y)

Learning rate set to 0.171007
0:	learn: 0.5565732	total: 990ms	remaining: 16m 29s
1:	learn: 0.4709713	total: 1.9s	remaining: 15m 45s
2:	learn: 0.4163461	total: 2.83s	remaining: 15m 40s
3:	learn: 0.3810529	total: 3.65s	remaining: 15m 10s
4:	learn: 0.3593899	total: 4.49s	remaining: 14m 53s
5:	learn: 0.3450608	total: 5.29s	remaining: 14m 37s
6:	learn: 0.3345586	total: 6.49s	remaining: 15m 20s
7:	learn: 0.3276775	total: 7.48s	remaining: 15m 27s
8:	learn: 0.3232856	total: 8.48s	remaining: 15m 34s
9:	learn: 0.3198081	total: 9.85s	remaining: 16m 14s
10:	learn: 0.3171255	total: 11s	remaining: 16m 24s
11:	learn: 0.3150632	total: 12.3s	remaining: 16m 49s
12:	learn: 0.3137856	total: 13.3s	remaining: 16m 52s
13:	learn: 0.3128735	total: 14.2s	remaining: 16m 38s
14:	learn: 0.3118237	total: 15.5s	remaining: 16m 55s
15:	learn: 0.3110099	total: 16.8s	remaining: 17m 10s
16:	learn: 0.3104157	total: 17.8s	remaining: 17m 8s
17:	learn: 0.3098701	total: 19.2s	remaining: 17m 27s
18:	learn: 0.3093128	total: 20

In [7]:
#train accuracy
catboost_clf.score(X, y)

0.9036375

In [8]:
yPred = catboost_clf.predict(X)

In [9]:
#various performance scores
acc = accuracy_score(y, yPred)
print("The accuracy is {}".format(acc))
  
prec = precision_score(y, yPred)
print("The precision is {}".format(prec))
  
rec = recall_score(y, yPred)
print("The recall is {}".format(rec))
  
f1 = f1_score(y, yPred)
print("The F1-Score is {}".format(f1))
  
MCC = matthews_corrcoef(y, yPred)
print("The Matthews correlation coefficient is{}".format(MCC))

The accuracy is 0.9036375
The precision is 0.8309543102910927
The recall is 0.1666375
The F1-Score is 0.27760482283976967
The Matthews correlation coefficient is0.34578420211929967


In [10]:
#ks score function
def ks_(data=None,target=None, prob=None):
    data['target0'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 10)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2%}'.format)
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2%}'.format)
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100
    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2%}'.format)
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2%}'.format)
    kstable.index = range(1,11)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    print(kstable)
    
    return 

In [11]:
#creating data frame to check ks score for train
ks=pd.DataFrame(catboost_clf.predict_proba(X))
del ks[0]

In [12]:
ks['target']=y

#ks table for the total train data
ks_(ks,'target',1)

        min_prob  max_prob   events  nonevents event_rate nonevent_rate  \
Decile                                                                    
1       0.232778  0.999084  32480.0    39520.0     40.60%         6.17%   
2       0.145954  0.232777  13080.0    58920.0     16.35%         9.21%   
3       0.107389  0.145954   8725.0    63275.0     10.91%         9.89%   
4       0.084882  0.107389   6605.0    65395.0      8.26%        10.22%   
5       0.070125  0.084881   5148.0    66852.0      6.44%        10.45%   
6       0.059452  0.070125   4238.0    67762.0      5.30%        10.59%   
7       0.050611  0.059452   3397.0    68603.0      4.25%        10.72%   
8       0.042464  0.050611   2825.0    69175.0      3.53%        10.81%   
9       0.033573  0.042464   2141.0    69859.0      2.68%        10.92%   
10      0.001902  0.033573   1361.0    70639.0      1.70%        11.04%   

       cum_eventrate cum_noneventrate    KS  
Decile                                       
1      

In [13]:
#creating data frame of cust ids with probabilities
X_t= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/X_t.csv')
test_cust= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_cust.csv')
result_model = pd.DataFrame(catboost_clf.predict_proba(X_t))
result=pd.concat([test_cust['cust_id'],result_model[1] ], axis=1)
result.rename(columns={1:'target'},inplace=True)

In [14]:
result

,cust_id,target
0,10146566,0.116405
1,10203613,0.050671
2,10322129,0.013319
3,10373212,0.091135
4,10487539,0.061369
...,...,...
179995,45739460,0.028422
179996,56006782,0.022940
179997,61138340,0.045600
179998,65715392,0.027553


In [15]:
#uploading results(probabilities)
result.to_csv('/content/drive/MyDrive/Colab Notebooks/results.csv',index=False)